# Setup

In [1]:
import os
import sys

sys.path.append(os.path.abspath("../.."))

In [2]:
from pathlib import Path

In [3]:
REPETITIONS = 5
FOLDS = 5

In [4]:
from src.experiment.sets.metric_sets import metrics_for_correlation_analysis
metrics_for_correlation_analysis

['macro_accuracy',
 'micro_accuracy',
 'macro_precision',
 'micro_precision',
 'macro_recall',
 'micro_recall',
 'macro_f1',
 'micro_f1',
 'MSE',
 'LogLoss',
 'AUNU',
 'micro_ROC-AUC']

In [5]:
from src.experiment.helpers.variables import report_output_root_dir
report_output_root_dir

WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output')

### Datasets

In [7]:
from src.experiment.sets.binary_balanced_datasets import binary_balanced_datasets
binary_balanced_datasets

{'all-in-one_sentiment_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/all-in-one_sentiment_balanced'),
  'classification_type': 'binary',
  'class_balance': 'balanced',
  'dataset_name': 'all-in-one_sentiment_balanced'},
 'amazon-reviews-0.25_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/amazon-reviews-0.25_balanced'),
  'classification_type': 'binary',
  'class_balance': 'balanced',
  'dataset_name': 'amazon-reviews-0.25_balanced'},
 'ceas_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/ceas_balanced'),
  'classification_type': 'binary',
  'class_balance': 'balanced',
  'dataset_name': 'ceas_balanced'},
 'colon-0.5_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/colon-0.5_balanced'),
  'classification_type': 'binary',
  'class_balance': 'balanced',
  'dataset_name': 'colon-0.5_balanced'},
 'fake-news_balanced': {'path': Wind

In [8]:
from src.experiment.sets.binary_imbalanced_datasets import binary_imbalanced_datasets
binary_imbalanced_datasets

{'all-in-one_sentiment_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/all-in-one_sentiment_imbalanced'),
  'classification_type': 'binary',
  'class_balance': 'imbalanced',
  'dataset_name': 'all-in-one_sentiment_imbalanced'},
 'amazon-reviews-0.25_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/amazon-reviews-0.25_imbalanced'),
  'classification_type': 'binary',
  'class_balance': 'imbalanced',
  'dataset_name': 'amazon-reviews-0.25_imbalanced'},
 'ceas_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/ceas_imbalanced'),
  'classification_type': 'binary',
  'class_balance': 'imbalanced',
  'dataset_name': 'ceas_imbalanced'},
 'colon-0.5_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/binary/colon-0.5_imbalanced'),
  'classification_type': 'binary',
  'class_balance': 'imbalanced',
  'dataset_name': 'colon-0.5_imbalanced'},
 'fa

In [9]:
from src.experiment.sets.multiclass_balanced_datasets import multiclass_balanced_datasets
multiclass_balanced_datasets

{'ag-news_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/ag-news_balanced'),
  'classification_type': 'multiclass',
  'class_balance': 'balanced',
  'dataset_name': 'ag-news_balanced'},
 'dbpedia-ontology_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/dbpedia-ontology_balanced'),
  'classification_type': 'multiclass',
  'class_balance': 'balanced',
  'dataset_name': 'dbpedia-ontology_balanced'},
 'gas-drift_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/gas-drift_balanced'),
  'classification_type': 'multiclass',
  'class_balance': 'balanced',
  'dataset_name': 'gas-drift_balanced'},
 'gtsrb-huelist_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/gtsrb-huelist_balanced'),
  'classification_type': 'multiclass',
  'class_balance': 'balanced',
  'dataset_name': 'gtsrb-huelist_balanced'},
 'irish-times_balanced': 

In [10]:
from src.experiment.sets.multiclass_imbalanced_datasets import multiclass_imbalanced_datasets
multiclass_imbalanced_datasets

{'ag-news_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/ag-news_imbalanced'),
  'classification_type': 'multiclass',
  'class_balance': 'imbalanced',
  'dataset_name': 'ag-news_imbalanced'},
 'dbpedia-ontology_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/dbpedia-ontology_imbalanced'),
  'classification_type': 'multiclass',
  'class_balance': 'imbalanced',
  'dataset_name': 'dbpedia-ontology_imbalanced'},
 'gas-drift_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/gas-drift_imbalanced'),
  'classification_type': 'multiclass',
  'class_balance': 'imbalanced',
  'dataset_name': 'gas-drift_imbalanced'},
 'gtsrb-huelist_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multiclass/gtsrb-huelist_imbalanced'),
  'classification_type': 'multiclass',
  'class_balance': 'imbalanced',
  'dataset_name': 'gtsrb-huelist_imbalan

In [6]:
from src.experiment.sets.multilabel_balanced_datasets import multilabel_balanced_datasets
multilabel_balanced_datasets

{'bookmarks_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/bookmarks_balanced'),
  'classification_type': 'multilabel',
  'class_balance': 'balanced',
  'dataset_name': 'bookmarks_balanced'},
 'emotions_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/emotions_balanced'),
  'classification_type': 'multilabel',
  'class_balance': 'balanced',
  'dataset_name': 'emotions_balanced'},
 'imdb_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/imdb_balanced'),
  'classification_type': 'multilabel',
  'class_balance': 'balanced',
  'dataset_name': 'imdb_balanced'},
 'mediamill_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/mediamill_balanced'),
  'classification_type': 'multilabel',
  'class_balance': 'balanced',
  'dataset_name': 'mediamill_balanced'},
 'ng20_balanced': {'path': WindowsPath('c:/VisualStudioRepositories/MU

In [11]:
from src.experiment.sets.multilabel_imbalanced_datasets import multilabel_imbalanced_datasets
multilabel_imbalanced_datasets

{'bookmarks_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/bookmarks_imbalanced'),
  'classification_type': 'multilabel',
  'class_balance': 'imbalanced',
  'dataset_name': 'bookmarks_imbalanced'},
 'emotions_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/emotions_imbalanced'),
  'classification_type': 'multilabel',
  'class_balance': 'imbalanced',
  'dataset_name': 'emotions_imbalanced'},
 'imdb_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/imdb_imbalanced'),
  'classification_type': 'multilabel',
  'class_balance': 'imbalanced',
  'dataset_name': 'imdb_imbalanced'},
 'mediamill_imbalanced': {'path': WindowsPath('c:/VisualStudioRepositories/MUSIC_DATA/datasets/multilabel/mediamill_imbalanced'),
  'classification_type': 'multilabel',
  'class_balance': 'imbalanced',
  'dataset_name': 'mediamill_imbalanced'},
 'ng20_imbalanced': {'path': WindowsPa

# Get report.json files and group by dataset

### Load all report files

In [12]:
report_files = list(report_output_root_dir.rglob('report.json'))
report_files = sorted(report_files, key=lambda x: x.name)
report_files

[WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_0/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_1/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_2/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_3/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_4/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/3/fold_0/report.json'),
 WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/i

In [ ]:
len(report_files) # 60 datasets * 4 models * 5 repetitions * 5 CV = 6000

6000

### Group by dataset

In [14]:
from collections import defaultdict
import json

# Group report files by "dataset name"
dataset_reports = defaultdict(list)
for file in report_files:
    with open(file, 'r') as f:
        data = json.load(f)
        dataset_name = data.get('dataset name')
        if dataset_name:
            dataset_reports[dataset_name].append(file)

# Convert defaultdict to regular dict for JSON serialization
grouped_by_dataset = dict(dataset_reports)

In [15]:
grouped_by_dataset

{'yelp_imbalanced': [WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_0/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_1/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_2/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_3/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/4/fold_4/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_analysis/output/multilabel/imbalanced/yelp_imbalanced/random_forest/3/fold_0/report.json'),
  WindowsPath('C:/VisualStudioRepositories/MUSIC_DATA/metric_an

In [ ]:
# 4 models * 5 repetitions * 5 CV = 100 reports per dataset
for dataset_name, files in grouped_by_dataset.items():
    print(f"{dataset_name}: {len(files)}")

yelp_imbalanced: 100
yeast_imbalanced: 100
tmc2007500_imbalanced: 100
scene_imbalanced: 100
nuswidevlad_imbalanced: 100
ng20_imbalanced: 100
mediamill_imbalanced: 100
imdb_imbalanced: 100
emotions_imbalanced: 100
bookmarks_imbalanced: 100
yelp_balanced: 100
yeast_balanced: 100
tmc2007500_balanced: 100
scene_balanced: 100
nuswidevlad_balanced: 100
ng20_balanced: 100
mediamill_balanced: 100
imdb_balanced: 100
emotions_balanced: 100
bookmarks_balanced: 100
volkert_imbalanced: 100
usps_imbalanced: 100
nyt-comments-april17_imbalanced: 100
news-category_imbalanced: 100
mfeat-karhunen_imbalanced: 100
irish-times_imbalanced: 100
gtsrb-huelist_imbalanced: 100
gas-drift_imbalanced: 100
dbpedia-ontology_imbalanced: 100
ag-news_imbalanced: 100
volkert_balanced: 100
usps_balanced: 100
nyt-comments-april17_balanced: 100
news-category_balanced: 100
mfeat-karhunen_balanced: 100
irish-times_balanced: 100
gtsrb-huelist_balanced: 100
gas-drift_balanced: 100
dbpedia-ontology_balanced: 100
ag-news_balanced

# Transform report data

### Create dfs

In [ ]:
# Get all metrics from reports grouped by dataset
metrics_grouped_by_dataset = dict()

for dataset_name, files in grouped_by_dataset.items():
    metrics_grouped_by_dataset[dataset_name] = []
    for file in files:
        with open(file, 'r') as f:
            data = json.load(f)
            metrics_grouped_by_dataset[dataset_name].append(data['metrics'])

metrics_grouped_by_dataset

In [ ]:
# Leave only metrics meant for correlation analysis
filtered_metrics_grouped_by_dataset = {
    dataset: [
        {k: m[k] for k in metrics_for_correlation_analysis if k in m}
        for m in metrics_list
    ]
    for dataset, metrics_list in metrics_grouped_by_dataset.items()
}

filtered_metrics_grouped_by_dataset

In [19]:
# Transform report data into DataFrames for each dataset
import pandas as pd

tables_by_dataset = {}

for dataset, metrics_list in filtered_metrics_grouped_by_dataset.items():
    df = pd.DataFrame(metrics_list)
    tables_by_dataset[dataset] = df.T

In [22]:
# Show example table for a specific dataset
tables_by_dataset['ng20_imbalanced']

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
macro_accuracy,0.966427,0.966490,0.966548,0.966596,0.966658,0.966079,0.966150,0.966220,0.966289,0.966354,...,0.971042,0.970489,0.970005,0.969468,0.968998,0.974239,0.973442,0.972746,0.972162,0.971543
micro_accuracy,0.966427,0.966490,0.966548,0.966596,0.966658,0.966079,0.966150,0.966220,0.966289,0.966354,...,0.971042,0.970489,0.970005,0.969468,0.968998,0.974239,0.973442,0.972746,0.972162,0.971543
macro_precision,0.639480,0.640867,0.642329,0.643637,0.644978,0.632019,0.633591,0.635040,0.636543,0.637985,...,0.696581,0.687723,0.679427,0.670934,0.663854,0.756992,0.740328,0.726379,0.715189,0.704807
micro_precision,0.714011,0.715986,0.717890,0.719704,0.721580,0.703800,0.705941,0.707972,0.710002,0.712020,...,0.759494,0.750621,0.742880,0.734632,0.727118,0.816048,0.801657,0.788786,0.778220,0.767788
macro_recall,0.518528,0.516654,0.514921,0.513052,0.511475,0.528138,0.526056,0.524015,0.522148,0.520221,...,0.579973,0.579388,0.578651,0.577352,0.577406,0.582949,0.581265,0.581143,0.580920,0.580389
micro_recall,0.583751,0.582060,0.580389,0.578625,0.577181,0.592682,0.590713,0.588849,0.587109,0.585343,...,0.643103,0.642130,0.641389,0.640154,0.639957,0.647141,0.645463,0.644932,0.644687,0.643754
macro_f1,0.571091,0.570408,0.569845,0.569122,0.568578,0.574244,0.573578,0.572868,0.572273,0.571604,...,0.631320,0.627560,0.623877,0.619624,0.616710,0.654942,0.648239,0.643292,0.639021,0.634784
micro_f1,0.642344,0.642114,0.641858,0.641500,0.641353,0.643479,0.643207,0.642940,0.642734,0.642497,...,0.696469,0.692150,0.688414,0.684147,0.680759,0.721845,0.715131,0.709643,0.705187,0.700322
MSE,0.030137,0.030047,0.029960,0.029876,0.029787,0.030638,0.030530,0.030424,0.030329,0.030234,...,0.024811,0.025497,0.026104,0.026757,0.027339,0.020809,0.021791,0.022662,0.023405,0.024172
LogLoss,1.685966,1.664863,1.644500,1.624472,1.604791,1.801609,1.777089,1.753054,1.730286,1.707846,...,0.940174,1.055955,1.161481,1.268405,1.365868,0.247651,0.413373,0.562731,0.695145,0.825669


In [28]:
# Show example table for a specific dataset
tables_by_dataset['yeast_imbalanced']

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
macro_accuracy,0.742192,0.742865,0.743110,0.743559,0.743997,0.739591,0.740153,0.740702,0.741031,0.741449,...,0.750649,0.748503,0.746324,0.744671,0.743517,0.762443,0.759468,0.756641,0.754548,0.752651
micro_accuracy,0.742192,0.742865,0.743110,0.743559,0.743997,0.739591,0.740153,0.740702,0.741031,0.741449,...,0.750649,0.748503,0.746324,0.744671,0.743517,0.762443,0.759468,0.756641,0.754548,0.752651
macro_precision,0.630187,0.631116,0.631520,0.632382,0.632963,0.626419,0.627120,0.627869,0.628794,0.629305,...,0.643493,0.639437,0.635112,0.633536,0.632043,0.665636,0.660897,0.654563,0.649757,0.646958
micro_precision,0.711242,0.711986,0.712300,0.713021,0.713575,0.708105,0.708609,0.709210,0.709961,0.710443,...,0.720675,0.717906,0.714928,0.712974,0.711777,0.735283,0.732208,0.728582,0.725527,0.722823
macro_recall,0.600208,0.600536,0.600347,0.600322,0.600216,0.599873,0.600026,0.600130,0.599899,0.599811,...,0.613905,0.612763,0.611200,0.610556,0.609392,0.620939,0.618783,0.617059,0.616353,0.615379
micro_recall,0.700986,0.701667,0.701870,0.702177,0.702532,0.698530,0.699081,0.699622,0.699703,0.700125,...,0.711878,0.709861,0.708014,0.706342,0.704886,0.722464,0.718843,0.716714,0.715635,0.713876
macro_f1,0.608965,0.609381,0.609261,0.609451,0.609448,0.608071,0.608271,0.608468,0.608583,0.608554,...,0.621601,0.619987,0.617679,0.616915,0.615843,0.631135,0.629175,0.626500,0.624670,0.623309
micro_f1,0.706077,0.706789,0.707047,0.707557,0.708011,0.703285,0.703813,0.704383,0.704795,0.705246,...,0.716250,0.713861,0.711454,0.709642,0.708315,0.728817,0.725464,0.722599,0.720547,0.718322
MSE,0.212936,0.212123,0.211396,0.210735,0.209997,0.217211,0.216299,0.215428,0.214659,0.213835,...,0.190052,0.194071,0.198014,0.201301,0.203991,0.166880,0.172437,0.177672,0.182041,0.186108
LogLoss,11.325486,11.173320,11.026773,10.887351,10.748401,12.148441,11.969871,11.799292,11.641858,11.481725,...,6.407240,7.207090,7.976103,8.651910,9.247226,1.673041,2.777240,3.812102,4.732816,5.600241


### Add headers

In [29]:
MODEL_HEADERS = ["random_forest", "logistic_regression", "knn", "decision_tree"]
REPETITION_HEADERS = [f"R{i}" for i in reversed(range(REPETITIONS))]
CV_HEADERS =  [f"CV{i}" for i in range(FOLDS)]

In [30]:
REPETITION_HEADERS

['R4', 'R3', 'R2', 'R1', 'R0']

In [31]:
CV_HEADERS

['CV0', 'CV1', 'CV2', 'CV3', 'CV4']

In [32]:
for dataset, table in tables_by_dataset.items():
    repeated_headers = CV_HEADERS * (table.shape[1] // len(CV_HEADERS))
    tables_by_dataset[dataset].columns = repeated_headers

In [33]:
for dataset, table in tables_by_dataset.items():
    n_reps = len(REPETITION_HEADERS)
    n_folds = FOLDS
    n_cols = table.shape[1]
  
    rep_headers = []
    fold_headers = []
    for rep in REPETITION_HEADERS:
        rep_headers.extend([rep] * n_folds)
        fold_headers.extend(CV_HEADERS)

    repeats = (n_cols + n_reps * n_folds - 1) // (n_reps * n_folds)
    rep_headers = (rep_headers * repeats)[:n_cols]
    fold_headers = (fold_headers * repeats)[:n_cols]
    table.columns = pd.MultiIndex.from_arrays([rep_headers, fold_headers])

In [34]:
for dataset, table in tables_by_dataset.items():
    n_models = len(MODEL_HEADERS)
    n_cols_per_model = REPETITIONS * FOLDS
    new_top_level = []
    for model in MODEL_HEADERS:
        new_top_level.extend([model] * n_cols_per_model)

    if len(new_top_level) < table.shape[1]:
        repeats = (table.shape[1] + n_cols_per_model - 1) // n_cols_per_model
        new_top_level = (MODEL_HEADERS * repeats)[:table.shape[1]]

    table.columns = pd.MultiIndex.from_arrays([new_top_level, table.columns.get_level_values(0), table.columns.get_level_values(1)])

### Display

In [52]:
tables_by_dataset["ceas_balanced"]

random_forest                                          \
                           R4                                           
                          CV0       CV1       CV2       CV3       CV4   
macro_accuracy       0.973519  0.973635  0.973773  0.973856  0.973929   
micro_accuracy       0.973519  0.973635  0.973773  0.973856  0.973929   
macro_precision      0.973628  0.973741  0.973877  0.973960  0.974031   
micro_precision      0.973519  0.973635  0.973773  0.973856  0.973929   
macro_recall         0.973519  0.973634  0.973773  0.973856  0.973929   
micro_recall         0.973519  0.973635  0.973773  0.973856  0.973929   
macro_f1             0.973517  0.973633  0.973771  0.973855  0.973928   
micro_f1             0.973519  0.973635  0.973773  0.973856  0.973929   
MSE                  0.024933  0.024842  0.024729  0.024654  0.024579   
LogLoss              1.553060  1.532576  1.513413  1.494063  1.475188   
AUNU                 0.976230  0.976527  0.976810  0.977087  0.977365   
micro_ROC-AUC        0.976230  0.976527  0.976810  0.977087  0.977365   

                                                                   ...  \
                       R3                                          ...   
                      CV0       CV1       CV2       CV3       CV4  ...   
macro_accuracy   0.972898  0.973069  0.973207  0.973377  0.973445  ...   
micro_accuracy   0.972898  0.973069  0.973207  0.973377  0.973445  ...   
macro_precision  0.973018  0.973188  0.973325  0.973493  0.973557  ...   
micro_precision  0.972898  0.973069  0.973207  0.973377  0.973445  ...   
macro_recall     0.972898  0.973069  0.973207  0.973377  0.973445  ...   
micro_recall     0.972898  0.973069  0.973207  0.973377  0.973445  ...   
macro_f1         0.972896  0.973067  0.973205  0.973376  0.973443  ...   
micro_f1         0.972898  0.973069  0.973207  0.973377  0.973445  ...   
MSE              0.025516  0.025371  0.025256  0.025126  0.025041  ...   
LogLoss          1.663232  1.639460  1.617353  1.595836  1.574184  ...   
AUNU             0.974617  0.974967  0.975291  0.975614  0.975922  ...   
micro_ROC-AUC    0.974617  0.974967  0.975291  0.975614  0.975922  ...   

                decision_tree                                          \
                           R1                                           
                          CV0       CV1       CV2       CV3       CV4   
macro_accuracy       0.978491  0.977852  0.977119  0.976614  0.976065   
micro_accuracy       0.978491  0.977852  0.977119  0.976614  0.976065   
macro_precision      0.978629  0.977991  0.977270  0.976775  0.976225   
micro_precision      0.978491  0.977852  0.977119  0.976614  0.976065   
macro_recall         0.978491  0.977852  0.977119  0.976613  0.976065   
micro_recall         0.978491  0.977852  0.977119  0.976614  0.976065   
macro_f1             0.978489  0.977850  0.977117  0.976612  0.976063   
micro_f1             0.978491  0.977852  0.977119  0.976614  0.976065   
MSE                  0.018673  0.019398  0.020214  0.020799  0.021420   
LogLoss              0.888829  0.990247  1.100640  1.186130  1.272165   
AUNU                 0.983469  0.981924  0.980254  0.979047  0.977844   
micro_ROC-AUC        0.983469  0.981924  0.980254  0.979047  0.977844   

                                                                   
                       R0                                          
                      CV0       CV1       CV2       CV3       CV4  
macro_accuracy   0.982422  0.981542  0.980703  0.979922  0.979235  
micro_accuracy   0.982422  0.981542  0.980703  0.979922  0.979235  
macro_precision  0.982540  0.981658  0.980827  0.980053  0.979373  
micro_precision  0.982422  0.981542  0.980703  0.979922  0.979235  
macro_recall     0.982422  0.981542  0.980703  0.979922  0.979234  
micro_recall     0.982422  0.981542  0.980703  0.979922  0.979235  
macro_f1         0.982420  0.981541  0.980702  0.979921  0.979233  
micro_f1         0.982422  0.9815

In [37]:
tables_by_dataset["ng20_imbalanced"]

random_forest                                          \
                           R4                                           
                          CV0       CV1       CV2       CV3       CV4   
macro_accuracy       0.966427  0.966490  0.966548  0.966596  0.966658   
micro_accuracy       0.966427  0.966490  0.966548  0.966596  0.966658   
macro_precision      0.639480  0.640867  0.642329  0.643637  0.644978   
micro_precision      0.714011  0.715986  0.717890  0.719704  0.721580   
macro_recall         0.518528  0.516654  0.514921  0.513052  0.511475   
micro_recall         0.583751  0.582060  0.580389  0.578625  0.577181   
macro_f1             0.571091  0.570408  0.569845  0.569122  0.568578   
micro_f1             0.642344  0.642114  0.641858  0.641500  0.641353   
MSE                  0.030137  0.030047  0.029960  0.029876  0.029787   
LogLoss              1.685966  1.664863  1.644500  1.624472  1.604791   
AUNU                 0.845202  0.846069  0.847213  0.848246  0.849240   
micro_ROC-AUC        0.870818  0.871870  0.872878  0.873900  0.874921   

                                                                   ...  \
                       R3                                          ...   
                      CV0       CV1       CV2       CV3       CV4  ...   
macro_accuracy   0.966079  0.966150  0.966220  0.966289  0.966354  ...   
micro_accuracy   0.966079  0.966150  0.966220  0.966289  0.966354  ...   
macro_precision  0.632019  0.633591  0.635040  0.636543  0.637985  ...   
micro_precision  0.703800  0.705941  0.707972  0.710002  0.712020  ...   
macro_recall     0.528138  0.526056  0.524015  0.522148  0.520221  ...   
micro_recall     0.592682  0.590713  0.588849  0.587109  0.585343  ...   
macro_f1         0.574244  0.573578  0.572868  0.572273  0.571604  ...   
micro_f1         0.643479  0.643207  0.642940  0.642734  0.642497  ...   
MSE              0.030638  0.030530  0.030424  0.030329  0.030234  ...   
LogLoss          1.801609  1.777089  1.753054  1.730286  1.707846  ...   
AUNU             0.839451  0.840625  0.841874  0.842904  0.844155  ...   
micro_ROC-AUC    0.865110  0.866339  0.867534  0.868642  0.869755  ...   

                decision_tree                                          \
                           R1                                           
                          CV0       CV1       CV2       CV3       CV4   
macro_accuracy       0.971042  0.970489  0.970005  0.969468  0.968998   
micro_accuracy       0.971042  0.970489  0.970005  0.969468  0.968998   
macro_precision      0.696581  0.687723  0.679427  0.670934  0.663854   
micro_precision      0.759494  0.750621  0.742880  0.734632  0.727118   
macro_recall         0.579973  0.579388  0.578651  0.577352  0.577406   
micro_recall         0.643103  0.642130  0.641389  0.640154  0.639957   
macro_f1             0.631320  0.627560  0.623877  0.619624  0.616710   
micro_f1             0.696469  0.692150  0.688414  0.684147  0.680759   
MSE                  0.024811  0.025497  0.026104  0.026757  0.027339   
LogLoss              0.940174  1.055955  1.161481  1.268405  1.365868   
AUNU                 0.890237  0.882677  0.875642  0.868704  0.863408   
micro_ROC-AUC        0.909589  0.902455  0.896105  0.889876  0.884957   

                                                                   
                       R0                                          
                      CV0       CV1       CV2       CV3       CV4  
macro_accuracy   0.974239  0.973442  0.972746  0.972162  0.971543  
micro_accuracy   0.974239  0.973442  0.972746  0.972162  0.971543  
macro_precision  0.756992  0.740328  0.726379  0.715189  0.704807  
micro_precision  0.816048  0.801657  0.788786  0.778220  0.767788  
macro_recall     0.582949  0.581265  0.581143  0.580920  0.580389  
micro_recall     0.647141  0.645463  0.644932  0.644687  0.643754  
macro_f1         0.654942  0.648239  0.643292  0.639021  0.634784  
micro_f1         0.721845  0.7151

# Calculate correlation matrixes

### Calculate

In [38]:
pearson_correlations_by_dataset_metrics = {
    dataset: table.T.corr(method='pearson')
    for dataset, table in tables_by_dataset.items()
}

In [39]:
spearman_correlations_by_dataset_metrics = {
    dataset: table.T.corr(method='spearman')
    for dataset, table in tables_by_dataset.items()
}

### Display examples

##### Pearson

In [58]:
pearson_correlations_by_dataset_metrics["gas-drift_balanced"]


,macro_accuracy,micro_accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_f1,micro_f1,MSE,LogLoss,AUNU,micro_ROC-AUC
macro_accuracy,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967537,0.972120,0.974166
micro_accuracy,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967536,0.972119,0.974165
macro_precision,0.999522,0.999522,1.000000,0.999522,0.999522,0.999522,0.999615,0.999522,-0.981463,-0.966453,0.969752,0.972560
micro_precision,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967536,0.972119,0.974166
macro_recall,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967537,0.972120,0.974166
micro_recall,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967536,0.972119,0.974166
macro_f1,0.999994,0.999994,0.999615,0.999994,0.999994,0.999994,1.000000,0.999994,-0.985341,-0.967360,0.971819,0.973955
micro_f1,1.000000,1.000000,0.999522,1.000000,1.000000,1.000000,0.999994,1.000000,-0.985820,-0.967536,0.972119,0.974166
MSE,-0.985820,-0.985820,-0.981463,-0.985820,-0.985820,-0.985820,-0.985341,-0.985820,1.000000,0.980542,-0.982249,-0.981391
LogLoss,-0.967537,-0.967536,-0.966453,-0.967536,-0.967537,-0.967536,-0.967360,-0.967536,0.980542,1.000000,-0.985417,-0.988473


In [41]:
pearson_correlations_by_dataset_metrics["ng20_imbalanced"]

,macro_accuracy,micro_accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_f1,micro_f1,MSE,LogLoss,AUNU,micro_ROC-AUC
macro_accuracy,1.000000,1.000000,0.968681,0.944596,0.657983,0.663928,0.924200,0.944039,-0.998456,-0.970165,0.994337,0.991473
micro_accuracy,1.000000,1.000000,0.968681,0.944596,0.657983,0.663928,0.924200,0.944039,-0.998456,-0.970165,0.994337,0.991473
macro_precision,0.968681,0.968681,1.000000,0.985585,0.487955,0.493349,0.821654,0.849229,-0.978117,-0.988660,0.983765,0.987914
micro_precision,0.944596,0.944596,0.985585,1.000000,0.376849,0.383547,0.749738,0.784597,-0.960988,-0.995309,0.966298,0.974053
macro_recall,0.657983,0.657983,0.487955,0.376849,1.000000,0.999587,0.895411,0.868638,-0.615547,-0.456949,0.593711,0.567849
micro_recall,0.663928,0.663928,0.493349,0.383547,0.999587,1.000000,0.898438,0.873049,-0.621851,-0.463551,0.601073,0.575273
macro_f1,0.924200,0.924200,0.821654,0.749738,0.895411,0.898438,1.000000,0.997777,-0.901769,-0.804897,0.888660,0.873911
micro_f1,0.944039,0.944039,0.849229,0.784597,0.868638,0.873049,0.997777,1.000000,-0.924533,-0.836108,0.913710,0.900544
MSE,-0.998456,-0.998456,-0.978117,-0.960988,-0.615547,-0.621851,-0.901769,-0.924533,1.000000,0.981996,-0.997392,-0.996226
LogLoss,-0.970165,-0.970165,-0.988660,-0.995309,-0.456949,-0.463551,-0.804897,-0.836108,0.981996,1.000000,-0.983531,-0.988238


##### Spearman

In [42]:
spearman_correlations_by_dataset_metrics["emotions_balanced"]

,macro_accuracy,micro_accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_f1,micro_f1,MSE,LogLoss,AUNU,micro_ROC-AUC
macro_accuracy,1.000000,1.000000,0.928410,0.926615,0.202533,0.242627,0.829693,0.915466,-0.938899,-0.872502,0.921773,0.931446
micro_accuracy,1.000000,1.000000,0.928410,0.926615,0.202533,0.242627,0.829693,0.915466,-0.938899,-0.872502,0.921773,0.931446
macro_precision,0.928410,0.928410,1.000000,0.994683,0.050549,0.090694,0.671731,0.801788,-0.940642,-0.780606,0.941638,0.932913
micro_precision,0.926615,0.926615,0.994683,1.000000,0.038980,0.078837,0.663018,0.790327,-0.940330,-0.787459,0.939454,0.933585
macro_recall,0.202533,0.202533,0.050549,0.038980,1.000000,0.996301,0.617390,0.423534,-0.092157,-0.231455,0.095818,0.116184
micro_recall,0.242627,0.242627,0.090694,0.078837,0.996301,1.000000,0.641744,0.456593,-0.134055,-0.275462,0.136570,0.156960
macro_f1,0.829693,0.829693,0.671731,0.663018,0.617390,0.641744,1.000000,0.948299,-0.663846,-0.748215,0.649037,0.671827
micro_f1,0.915466,0.915466,0.801788,0.790327,0.423534,0.456593,0.948299,1.000000,-0.780786,-0.861434,0.773237,0.790219
MSE,-0.938899,-0.938899,-0.940642,-0.940330,-0.092157,-0.134055,-0.663846,-0.780786,1.000000,0.823426,-0.987291,-0.985119
LogLoss,-0.872502,-0.872502,-0.780606,-0.787459,-0.231455,-0.275462,-0.748215,-0.861434,0.823426,1.000000,-0.822514,-0.845689


In [44]:
spearman_correlations_by_dataset_metrics["ng20_imbalanced"]

,macro_accuracy,micro_accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_f1,micro_f1,MSE,LogLoss,AUNU,micro_ROC-AUC
macro_accuracy,1.000000,1.000000,0.733897,0.705275,0.815494,0.819418,0.818182,0.823066,-0.982394,-0.737990,0.952907,0.940810
micro_accuracy,1.000000,1.000000,0.733897,0.705275,0.815494,0.819418,0.818182,0.823066,-0.982394,-0.737990,0.952907,0.940810
macro_precision,0.733897,0.733897,1.000000,0.984758,0.512583,0.514095,0.519304,0.520888,-0.818626,-0.991887,0.801056,0.850333
micro_precision,0.705275,0.705275,0.984758,1.000000,0.468227,0.472667,0.472919,0.477252,-0.792811,-0.982274,0.782178,0.834239
macro_recall,0.815494,0.815494,0.512583,0.468227,1.000000,0.991923,0.998020,0.991083,-0.736466,-0.519352,0.698002,0.661854
micro_recall,0.819418,0.819418,0.514095,0.472667,0.991923,1.000000,0.992031,0.997420,-0.737114,-0.518368,0.702502,0.665035
macro_f1,0.818182,0.818182,0.519304,0.472919,0.998020,0.992031,1.000000,0.993735,-0.739550,-0.522412,0.699166,0.666067
micro_f1,0.823066,0.823066,0.520888,0.477252,0.991083,0.997420,0.993735,1.000000,-0.741602,-0.522544,0.700210,0.668215
MSE,-0.982394,-0.982394,-0.818626,-0.792811,-0.736466,-0.737114,-0.739550,-0.741602,1.000000,0.822490,-0.969757,-0.976658
LogLoss,-0.737990,-0.737990,-0.991887,-0.982274,-0.519352,-0.518368,-0.522412,-0.522544,0.822490,1.000000,-0.811497,-0.861158


# Save

In [54]:
all_datasets = binary_balanced_datasets | binary_imbalanced_datasets | multiclass_balanced_datasets | multiclass_imbalanced_datasets | multilabel_balanced_datasets | multilabel_imbalanced_datasets

In [55]:
def save_correlation_matrices(correlations_by_dataset, corr_type, filename, datasets):
    for dataset_name, corr_matrix in correlations_by_dataset.items():
        dataset_info = datasets[dataset_name]
        output_dir = report_output_root_dir / dataset_info["classification_type"] / dataset_info["class_balance"] / dataset_info["dataset_name"]
        output_path = output_dir / filename

        output = {
            "dataset_name": dataset_info["dataset_name"],
            "classification_type": dataset_info["classification_type"],
            "class_balance": dataset_info["class_balance"],
            "correlation_type": corr_type,
            "correlation_matrix": corr_matrix.to_dict()
        }

        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(output, f, indent=2)

In [56]:
save_correlation_matrices(pearson_correlations_by_dataset_metrics, "pearson", "pearson_correlation.json", all_datasets)
save_correlation_matrices(spearman_correlations_by_dataset_metrics, "spearman", "spearman_correlation.json", all_datasets)